<!-- <img  src="https://swan.web.cern.ch/sites/swan.web.cern.ch/files/pictures/logo_swan_letters.png" alt="SWAN" style="float: left; width: 15%; margin-right: 5%; margin-left: 17%; margin-top: 1.0em; margin-bottom: 2.0em;">
<img src="https://spark.apache.org/images/spark-logo-trademark.png" alt="EP-SFT" style="float: left; width: 25%; margin-right: 0%; margin-left: 0%; margin-bottom: 2.0em;">
<img src="https://cms-docdb.cern.ch/cgi-bin/PublicDocDB/RetrieveFile?docid=3045&filename=CMSlogo_color_label_1024_May2014.png&version=3" alt="CMS" style="float: left; width: 12%; margin-left: 5%; margin-right: 5%; margin-bottom: 2.0em;"> -->
<p style="clear: both;">
<div style="text-align:center"><h1>CMS H&#8594;µµ analysis  
     <br> with Coffea package from Fermilab</h1></div>
<div style="text-align:center"><i>Author: Dmitry Kondratyev, based on example code by Lindsey Gray</i></div>
<hr style="border-top-width: 4px; border-top-color: #34609b;">

# Search for Higgs boson decaying into two muons

This code uses awkward array toolset, and utilizing Coffea [histograms](https://coffeateam.github.io/coffea/modules/coffea.hist.html).
This also shows the analysis object syntax implemented by Coffea [JaggedCandidateArray](https://coffeateam.github.io/coffea/api/coffea.analysis_objects.JaggedCandidateMethods.html), and the usage of custom [accumulators](https://coffeateam.github.io/coffea/api/coffea.processor.AccumulatorABC.html) other than histograms.  Further, it introduces the [processor](https://coffeateam.github.io/coffea/api/coffea.processor.ProcessorABC.html) concept and the interface to apache spark.

#### SWAN env: LCG96 Python3 stack and Cloud Containers cluster

In [ ]:
# Run this cell if you do not have coffea installed (e.g. on SWAN with LCG 96Python3 stack)
!pip install --user --upgrade coffea

# spark.jars.packages doesnt work with Spark 2.4 with kubernetes
!wget -N https://repo1.maven.org/maven2/edu/vanderbilt/accre/laurelin/0.5.1/laurelin-0.5.1.jar && \
wget -N https://repo1.maven.org/maven2/org/apache/logging/log4j/log4j-api/2.11.2/log4j-api-2.11.2.jar && \
wget -N https://repo1.maven.org/maven2/org/apache/logging/log4j/log4j-core/2.11.2/log4j-core-2.11.2.jar && \
wget -N https://repo1.maven.org/maven2/org/lz4/lz4-java/1.5.1/lz4-java-1.5.1.jar && \
wget -N https://repo1.maven.org/maven2/org/tukaani/xz/1.2/xz-1.2.jar
                    
!mkdir output

In [ ]:
# Run this cell before establishing spark connection

import os
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + ':' + '/usr/local/lib/python3.6/site-packages'

In [ ]:
import time
import coffea
print("Coffea version: ", coffea.__version__)

from coffea import hist, util
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
from coffea.lookup_tools import extractor, dense_lookup, txt_converters, rochester_lookup
from coffea.lumi_tools import LumiMask

import awkward
import uproot
import numpy as np
import numba
import pandas as pd

In [ ]:
import glob

from datasets import datasets, lumi_data


year = "2016"
paths = datasets[year]
# TODO: add possibility to load of files through XrootD

debug = True

if debug:
    samples = [
        'data_G', 
        'dy', 
        'dy_m105_160_amc',
        'ttjets_dl',      
        'ewk_lljj_mll50_mjj120',
        'ewk_lljj_mll105_160',
        'ggh_amcPS',
        'vbf_amcPS',
        
#         'st_tw_top',
#         'st_tw_antitop',
#         'ww_2l2nu',
#         'wz_3lnu',
#         'www',
#         'wwz',
#         'wzz',
#         'zzz',
        
    ]
else:
    samples = paths.keys()

fileset = {}
metadata = {}

data_samples = [s for s in samples if 'data' in s]
signal_samples = ['ggh_amcPS', 'vbf_amcPS']
main_bkg_samples = ['dy_m105_160_amc', 'ttjets_dl', 'ewk_lljj_mll105_160']
datasets_to_save_unbin = data_samples + signal_samples + main_bkg_samples

lumi = 1
data_entries = 0

# Find ROOT files in local directories
for sample in samples:
    metadata[sample] = {}
    all_files = []
    all_files = glob.glob(paths[sample]+'*root')
    
    if debug:
        all_files = [all_files[0]]
#         if 'data' in sample:
#             all_files = all_files[0:10]
#         else:
#             all_files = [all_files[0]]
     
#     server = 'root://eoscms.cern.ch/'
    server = ''
            
    if 'data' in sample:
        for f in all_files:
            tree = uproot.open(server+f)['Events']
            data_entries += tree.numentries
    else:
        sumGenWgts = 0
        nGenEvts = 0
        for f in all_files:
            tree = uproot.open(server+f)['Runs']
            if 'NanoAODv6' in paths[sample]:
                sumGenWgts += tree.array('genEventSumw_')[0]
                nGenEvts += tree.array('genEventCount_')[0]
            else:
                sumGenWgts += tree.array('genEventSumw')[0]
                nGenEvts += tree.array('genEventCount')[0]
        metadata[sample]['sumGenWgts'] = sumGenWgts
        metadata[sample]['nGenEvts'] = nGenEvts
        
    fileset[sample] = {
        'files': [server+f for f in all_files],
        'treename': 'Events'
    }

print(f"Loading {data_entries} of {year} data events")
lumi = lumi_data[year]['lumi']*data_entries/lumi_data[year]['events']
print(f"This is ~ {data_entries/lumi_data[year]['events']*100}% of {year} data.")
print(f"Integrated luminosity {lumi}/pb")

In [ ]:

puLookup = util.load('data/pileup/puLookup.coffea')
muSFFileList = [{'id'   : ("data/muon_sf/EfficienciesStudies_2016_legacy_rereco_rootfiles_RunBCDEF_SF_ID.root", "NUM_TightID_DEN_genTracks_eta_pt"),
                 'iso'   : ("data/muon_sf/EfficienciesStudies_2016_legacy_rereco_rootfiles_RunBCDEF_SF_ISO.root", "NUM_TightRelIso_DEN_TightIDandIPCut_eta_pt"),
                 'trig'  : ("data/muon_sf/EfficienciesStudies_2016_trigger_EfficienciesAndSF_RunBtoF.root", "IsoMu24_OR_IsoTkMu24_PtEtaBins/abseta_pt_ratio"),
                 'scale' : 19.656062760/35.882515396},
                {'id'     : ("data/muon_sf/EfficienciesStudies_2016_legacy_rereco_rootfiles_RunGH_SF_ID.root", "NUM_TightID_DEN_genTracks_eta_pt"),
                 'iso'   : ("data/muon_sf/EfficienciesStudies_2016_legacy_rereco_rootfiles_RunGH_SF_ISO.root", "NUM_TightRelIso_DEN_TightIDandIPCut_eta_pt"),
                 'trig'  : ("data/muon_sf/EfficienciesStudies_2016_trigger_EfficienciesAndSF_RunGtoH.root", "IsoMu24_OR_IsoTkMu24_PtEtaBins/abseta_pt_ratio"),
                 'scale' : 16.226452636/35.882515396}]
# TODO: check scale

zpt_weights_file = "data/zpt/zpt_weights.histo.json"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def apply_roccor(rochester, isData, muons):
    muons = muons.compact()
    corrections = muons.pt.ones_like()  
    if isData:
        corrections = rochester.kScaleDT(muons.charge, muons.pt, muons.eta, muons.phi)      
    else:
        mc_rand = np.random.rand(*muons.pt.flatten().shape)
        mc_rand = awkward.JaggedArray.fromoffsets(muons.pt.offsets, mc_rand)
        hasgen = ~np.isnan(muons.matched_gen.pt.fillna(np.nan))
        mc_rand = awkward.JaggedArray.fromoffsets(hasgen.offsets, mc_rand)._content

        mc_kspread = rochester.kSpreadMC(muons.charge[hasgen], muons.pt[hasgen], muons.eta[hasgen], muons.phi[hasgen],
                                         muons.matched_gen.pt[hasgen])
        mc_ksmear = rochester.kSmearMC(muons.charge[~hasgen], muons.pt[~hasgen],muons.eta[~hasgen],muons.phi[~hasgen],
                                       muons.nTrackerLayers[~hasgen], mc_rand[~hasgen])
        corrections = np.ones_like(muons.pt.flatten())
        corrections[hasgen.flatten()] = mc_kspread.flatten()
        corrections[~hasgen.flatten()] = mc_ksmear.flatten() 
    return corrections

def p4_sum(obj1, obj2):
    assert(obj1.shape==obj2.shape)
    px = np.zeros(obj1.shape[0])
    py = np.zeros(obj1.shape[0])
    pz = np.zeros(obj1.shape[0])
    e = np.zeros(obj1.shape[0])
    
    for obj in [obj1, obj2]:
        px_ = obj.pt*np.cos(obj.phi)
        py_ = obj.pt*np.sin(obj.phi)
        pz_ = obj.pt*np.sinh(obj.eta)
        e_  = np.sqrt(px_**2 + py_**2 + pz_**2 + obj.mass**2)
        px = px + px_
        py = py + py_
        pz = pz + pz_
        e = e + e_
        
    pt = np.sqrt(px**2 + py**2)
    eta = np.arcsinh(pz / pt)
    phi = np.arctan2(py, px)
    mass = np.sqrt(e**2 - px**2 - py**2 - pz**2)    
    return pt, eta, phi, mass


        
def get_regions(mass):
    regions = {
        "z-peak": ((mass>70) & (mass<110)),
        "h-sidebands": ((mass>110) & (mass<115)) | ((mass>135) & (mass<150)),
        "h-peak": ((mass>115) & (mass<135)),
    }
    return regions


class Timer(object):
    def __init__(self, name="t"):
        self.name = name
        self.time_dict = {}
        self.last_checkpoint = time.time()    
        
    def update(self):
        self.last_checkpoint = time.time()
        
    def add_checkpoint(self, comment):
        now = time.time()
        dt = now - self.last_checkpoint
        if comment in self.time_dict.keys():
            self.time_dict[comment] += dt
        else:
            self.time_dict[comment] = dt
        self.last_checkpoint = now
        
    def summary(self):
        columns = ["Action", "Time (s)", "% CPU"]
        summary = pd.DataFrame(columns=columns)        
        total_time = sum(list(self.time_dict.values()))
        summary[columns[0]] = np.array(list(self.time_dict.keys()))
        summary[columns[1]] = np.round( np.array(list(self.time_dict.values())), 5)
        summary[columns[2]] = np.round( 100*np.array(list(self.time_dict.values()))/total_time, 3)

        print('-'*50)
        print(f'Summary of {self.name} timer:')
        print('-'*50)
        print(summary)
        print('-'*50)
        print(f'Total time: {total_time}')
        print('='*50)
        print()
        


In [ ]:
# Look at ProcessorABC documentation to see the expected methods and what they are supposed to do
# https://coffeateam.github.io/coffea/api/coffea.processor.ProcessorABC.html
class DimuonProcessor(processor.ProcessorABC):
    def __init__(self, mass_window=[70,150], do_roccor=True, evaluate_dnn=True, do_timer=False): 
        from configuration import parameters
        from variables import variables
        year = '2016'
        self.mass_window = mass_window
        self.do_roccor = do_roccor
        self.evaluate_dnn = evaluate_dnn
        self.parameters = {k:v[year] for k,v in parameters.items()}
        self.timer = Timer('global') if do_timer else None
        
        self._columns = self.parameters["proc_columns"]

        dataset_axis = hist.Cat("dataset", "")
        region_axis = hist.Cat("region", " ") # Z-peak, Higgs SB, Higgs peak
        channel_axis = hist.Cat("channel", " ") # ggh or VBF       
        accumulators = {}
        
        ### Prepare accumulators for binned output ###
        
        for v in variables:
            if 'dimuon_mass' in v.name:
                axis = hist.Bin(v.name, v.caption, v.nbins, self.mass_window[0], self.mass_window[1])
            else:
                axis = hist.Bin(v.name, v.caption, v.nbins, v.xmin, v.xmax)
            accumulators[v.name] = hist.Hist("Counts", dataset_axis, region_axis, channel_axis, axis)
            
        accumulators['cutflow'] = processor.defaultdict_accumulator(int)

        ### Prepare accumulators for unbinned output ###
        
        self.vars_unbin = ['event', 'event_weight', 'dimuon_mass', 'dimuon_pt', 'dimuon_eta',\
                           'dimuon_dEta', 'mu1_pt', 'mu2_pt']
        self.regions = ['z-peak', 'h-sidebands', 'h-peak']
        self.channels = ['ggh_01j', 'ggh_2j', 'vbf']
        
        self.dont_fill = {
            'z-peak': ['dy_m105_160_amc', 'ewk_lljj_mll105_160'],
            'h-sidebands': ['dy', 'ewk_lljj_mll50_mjj120'],
            'h-peak': ['dy', 'ewk_lljj_mll50_mjj120'],
        }
        
        for p in datasets_to_save_unbin:
            for v in self.vars_unbin:
                for c in self.channels:
                    for r in self.regions:
                        if 'z-peak' in r: continue # don't need unbinned data for Z-peak
                        accumulators[f'{v}_unbin_{p}_c_{c}_r_{r}'] = processor.column_accumulator(np.ndarray([]))
                        # have to encode everything into the name because having multiple axes isn't possible
        
        self._accumulator = processor.dict_accumulator(accumulators)
    
        ### --------------------------------------- ###
        
        mu_id_vals = 0
        mu_id_err = 0
        mu_iso_vals = 0
        mu_iso_err = 0
        mu_trig_vals = 0
        mu_trig_err = 0

        for scaleFactors in muSFFileList:
            id_file = uproot.open(scaleFactors['id'][0])
            iso_file = uproot.open(scaleFactors['iso'][0])
            trig_file = uproot.open(scaleFactors['trig'][0])
            
            mu_id_vals += id_file[scaleFactors['id'][1]].values * scaleFactors['scale']
            mu_id_err += id_file[scaleFactors['id'][1]].variances**0.5 * scaleFactors['scale']
            mu_id_edges = id_file[scaleFactors['id'][1]].edges

            mu_iso_vals += iso_file[scaleFactors['iso'][1]].values * scaleFactors['scale']
            mu_iso_err += iso_file[scaleFactors['iso'][1]].variances**0.5 * scaleFactors['scale']
            mu_iso_edges = iso_file[scaleFactors['iso'][1]].edges

            mu_trig_vals += trig_file[scaleFactors['trig'][1]].values * scaleFactors['scale']
            mu_trig_err += trig_file[scaleFactors['trig'][1]].variances**0.5 * scaleFactors['scale']
            mu_trig_edges = trig_file[scaleFactors['trig'][1]].edges

        self.mu_id_sf = dense_lookup.dense_lookup(mu_id_vals, mu_id_edges)
        self.mu_id_err = dense_lookup.dense_lookup(mu_id_err, mu_id_edges)
        self.mu_iso_sf = dense_lookup.dense_lookup(mu_iso_vals, mu_iso_edges)
        self.mu_iso_err = dense_lookup.dense_lookup(mu_iso_err, mu_iso_edges)
        self.mu_trig_sf = dense_lookup.dense_lookup(mu_trig_vals, mu_trig_edges)
        self.mu_trig_err = dense_lookup.dense_lookup(mu_trig_err, mu_trig_edges)    

        self.extractor = extractor()
        self.extractor.add_weight_sets([f"* * {zpt_weights_file}"])
        self.extractor.finalize()
        self.evaluator = self.extractor.make_evaluator()
        if '2016' in year:
            self.zpt_path = 'zpt_weights/2016_value'
        else:
            self.zpt_path = 'zpt_weights/2017_value'
        self.evaluator[self.zpt_path]._axes = self.evaluator[self.zpt_path]._axes[0]            
#         Have to do the last line because of a bug in lookup_tools
#         For 1-dimensional histograms, _axes is a tuple, and np.searchsorted doesn't understand it
#         https://github.com/CoffeaTeam/coffea/blob/2650ad7657094f6e50ebf962a1fc1763cd2c6601/coffea/lookup_tools/dense_lookup.py#L37
#         TODO: tell developers?

        rochester_data = txt_converters.convert_rochester_file(self.parameters["roccor_file"], loaduncs=True)
        self.rochester = rochester_lookup.rochester_lookup(rochester_data)
    
    @property
    def accumulator(self):
        return self._accumulator
    
    @property
    def columns(self):
        return self._columns
    
    def process(self, df):
        # TODO: Properly calculate integrated luminosity
        # TODO: Add FSR recovery
        # TODO: verify PU weigths (ask at https://github.com/dnoonan08/TTGamma_LongExercise)
        # TODO: generate lepton SF for 2017/2018
        # TODO: NNLOPS reweighting (ggH)
        # TODO: L1 prefiring weights
        # TODO: btag sf
        # TODO: jet PU ID sf (copy-paste my code from hepaccelerate)
        # TODO: compute dimuon_costhetaCS, dimuon_phiCS
        # TODO: compute nsoftjets
        # TODO: JEC, JER
        # TODO: event-by-event mass resolution and calibration
        # TODO: kinematic variables of multimuon-multijet system
        # TODO: filter by event number to make sure DNN is evaluated only on test events
        # TODO: Add systematic uncertainties
        
        if self.timer:
            self.timer.update()
            
        output = self.accumulator.identity()
        dataset = df.metadata['dataset']
        isData = 'data' in dataset
            
        nEvts = df.shape[0]

        if isData:
            lumi_info = LumiMask(self.parameters['lumimask'])
            lumimask = lumi_info(df.run.flatten(), df.luminosityBlock.flatten())
            event_weight = np.ones(nEvts)
        else:    
            lumimask = np.ones(nEvts, dtype=bool)
            genweight = df.genWeight.flatten()
            pu_weight = puLookup(dataset, df.Pileup.nTrueInt)
            event_weight = genweight*pu_weight        
        
        hlt = np.zeros(nEvts, dtype=bool)
        for hlt_path in self.parameters['hlt']:
            hlt = hlt | df.HLT[hlt_path]

        mask = hlt & lumimask
    
        # Filter 0: HLT & lumimask
        #--------------------------------#    
        df = df[mask]
        event_weight = event_weight[mask]
        if self.timer:
            self.timer.add_checkpoint("Applied HLT and lumimask")
        #--------------------------------# 
        
        pass_event_flags = np.ones(df.shape[0], dtype=bool)
        for flag in self.parameters["event_flags"]:
            pass_event_flags = pass_event_flags & df.Flag[flag]
        
        pass_muon_flags = np.ones(df.shape[0], dtype=bool)
        for flag in self.parameters["muon_flags"]:
            pass_muon_flags = pass_muon_flags & df.Muon[flag]
        
        muons = df.Muon[(df.Muon.pt > self.parameters["muon_pt_cut"]) &\
                        (abs(df.Muon.eta) < self.parameters["muon_eta_cut"]) &\
                        (df.Muon.pfRelIso04_all < self.parameters["muon_iso_cut"]) &\
                        df.Muon[self.parameters["muon_id"]] & pass_muon_flags]
              
        two_os_muons = ((muons.counts == 2) & (muons['charge'].prod() == -1))
        
        electrons = df.Electron[(df.Electron.pt > self.parameters["electron_pt_cut"]) &\
                                     (abs(df.Electron.eta) < self.parameters["electron_eta_cut"]) &\
                                     (df.Electron[self.parameters["electron_id"]] == 1)]
        
        electron_veto = (electrons.counts==0)
        good_pv = (df.PV.npvsGood > 0)
            
        event_filter = (pass_event_flags & two_os_muons & electron_veto & good_pv).flatten()
        
        
        # Filter 1: Event selection
        #--------------------------------#    
        df = df[event_filter]
        muons = muons[event_filter]
        event_weight = event_weight[event_filter]
        if self.timer:
            self.timer.add_checkpoint("Applied preselection")
        #--------------------------------# 

        mu1 = muons[muons.pt.argmax()]
        mu2 = muons[muons.pt.argmin()]
        if self.do_roccor:
            mu1 = JaggedCandidateArray.candidatesfromcounts(
                np.ones(mu1.pt.shape),
                pt=mu1.pt.content*apply_roccor(self.rochester, isData, mu1).flatten(),
                eta=mu1.eta.content,
                phi=mu1.phi.content,
                mass=mu1.mass.content,
                pfRelIso04_all=mu1.pfRelIso04_all.content
            )
            mu2 = JaggedCandidateArray.candidatesfromcounts(
                np.ones(mu2.pt.shape),
                pt=mu2.pt.content*apply_roccor(self.rochester, isData, mu2).flatten(),
                eta=mu2.eta.content,
                phi=mu2.phi.content,
                mass=mu2.mass.content,
                pfRelIso04_all=mu2.pfRelIso04_all.content
            )
            if self.timer:
                self.timer.add_checkpoint("Applied Rochester")

        # correct dimuon kinematics..
        dimuon_pt, dimuon_eta, dimuon_phi, dimuon_mass = p4_sum(mu1, mu2)
        
        # gives wrong dimuon mass!
#         dimuons = JaggedCandidateArray.candidatesfromcounts(
#             np.ones(dimuon_pt.shape),
#             pt=dimuon_pt.content,
#             eta=dimuon_eta.content,
#             phi=dimuon_phi.content,
#             mass=dimuon_mass.content,
#         )

        if 'dy' in dataset:
            zpt_weights = self.evaluator[self.zpt_path](dimuon_pt).flatten()
            event_weight = event_weight*zpt_weights

        mu_pass_leading_pt = muons[(muons.pt > self.parameters["muon_leading_pt"]) &\
                                   (muons.pfRelIso04_all < self.parameters["muon_trigmatch_iso"]) &\
                                   muons[self.parameters["muon_trigmatch_id"]]]
        trig_muons = df.TrigObj[df.TrigObj.id == 13]
        muTrig = mu_pass_leading_pt.cross(trig_muons, nested = True)
        matched = (muTrig.i0.delta_r(muTrig.i1) < self.parameters["muon_trigmatch_dr"])

        # at least one muon matched with L3 object, and that muon passes pt, iso and id cuts
        trig_matched = (mu_pass_leading_pt[matched.any()].counts>0)

        
        dimuon_filter = ((mu1.pt>self.parameters["muon_leading_pt"]) & trig_matched &\
                         (dimuon_mass > self.mass_window[0]) & (dimuon_mass < self.mass_window[1])).flatten()

        if not isData:
            muID = self.mu_id_sf(muons.eta.compact(), muons.pt.compact())
            muIso = self.mu_iso_sf(muons.eta.compact(), muons.pt.compact())
            muTrig = self.mu_iso_sf(abs(muons.eta.compact()), muons.pt.compact())
            muSF = (muID*muIso*muTrig).prod()
            event_weight = event_weight*muSF
#             muIDerr = self.mu_id_err(muons.eta, muons.pt)
#             muIsoerr = self.mu_iso_err(muons.eta, muons.pt)
#             muTrigerr = self.mu_iso_err(abs(muons.eta), muons.pt)
#             muSF_up = ((muID + muIDerr) * (muIso + muIsoerr) * (muTrig + muTrigerr)).prod()
#             muSF_down = ((muID - muIDerr) * (muIso - muIsoerr) * (muTrig - muTrigerr)).prod() 

    
        # Filter 2: Dimuon pair selection
        #--------------------------------#
        df = df[dimuon_filter]   
        mu1 = mu1[dimuon_filter] 
        mu2 = mu2[dimuon_filter] 
        muons = muons[dimuon_filter]
        dimuon_pt = dimuon_pt[dimuon_filter]
        dimuon_eta = dimuon_eta[dimuon_filter]
        dimuon_phi = dimuon_phi[dimuon_filter]
        dimuon_mass = dimuon_mass[dimuon_filter]
        event_weight = event_weight[dimuon_filter]
        if self.timer:
            self.timer.add_checkpoint("Applied dimuon cuts")
        #--------------------------------#   

                            
        mujet = df.Jet.cross(muons, nested=True)
        deltar_mujet = mujet.i0.delta_r(mujet.i1)
        deltar_mujet_ok =  (deltar_mujet > self.parameters["min_dr_mu_jet"]).all()
        
        # 2016: loose jetId, loose piId 
        if "loose" in self.parameters["jet_id"]:
            jet_id = (df.Jet.jetId >= 1)
        elif "tight" in self.parameters["jet_id"]:
            jet_id = (df.Jet.jetId >= 1)
        else:
            jet_id = df.Jet.ones_like()
            
        if "loose" in self.parameters["jet_puid"]:
            jet_puid = (((df.Jet.puId >= 4) & (df.Jet.pt < 50)) | (df.Jet.pt > 50))
        else:
            jet_puid = df.Jet.ones_like()
            
        jet_selection = ((df.Jet.pt > self.parameters["jet_pt_cut"]) &\
                         (abs(df.Jet.eta) < self.parameters["jet_eta_cut"]) &\
                         jet_id & jet_puid & (df.Jet.qgl > -2) & deltar_mujet_ok )
        
        # TODO: check jet selections
        
        jets = df.Jet[jet_selection]
        # check if there should be this eta<2.5 cut 
        nBtagLoose = jets[(jets.btagDeepB>self.parameters["btag_loose_wp"]) & (abs(jets.eta)<2.5)].counts
        nBtagMedium = jets[(jets.btagDeepB>self.parameters["btag_medium_wp"])  & (abs(jets.eta)<2.5)].counts
        jet_filter = ((nBtagLoose<2)&(nBtagMedium<1))

        # Filter 3: Jet filter
        #--------------------------------#
        df = df[jet_filter]   
        mu1 = mu1[jet_filter] 
        mu2 = mu2[jet_filter] 
        dimuon_pt = dimuon_pt[jet_filter]
        dimuon_eta = dimuon_eta[jet_filter]
        dimuon_phi = dimuon_phi[jet_filter]
        dimuon_mass = dimuon_mass[jet_filter]
        jets = jets[jet_filter]
        jet_selection = jet_selection[jet_filter]
        event_weight = event_weight[jet_filter]
        if self.timer:
            self.timer.add_checkpoint("Applied jet cuts")

        #--------------------------------#

        
        # In the computations below I'm trying to keep the size of objects in first dimension the same
        # as it is in the previous step, in order to be able to apply event_weight similarly for all variables
        
        one_jet = (jet_selection.any() & (jets.counts>0))
        two_jets = (jet_selection.any() & (jets.counts>1))
        
        category = np.empty(df.shape[0], dtype=object)
        category[~two_jets] = 'ggh_01j'
        
        jet1_mask = np.zeros(len(event_weight))
        jet1_mask[one_jet] = 1
        jet1 = JaggedCandidateArray.candidatesfromcounts(
            jet1_mask,
            pt=jets[one_jet,0].pt.flatten(),
            eta=jets[one_jet,0].eta.flatten(),
            phi=jets[one_jet,0].phi.flatten(),
            mass=jets[one_jet,0].mass.flatten(),
            qgl=jets[one_jet,0].qgl.flatten()
        )
        
        jet2_mask = np.zeros(len(event_weight))
        jet2_mask[two_jets] = 1
        jet2 = JaggedCandidateArray.candidatesfromcounts(
            jet2_mask,
            pt=jets[two_jets,1].pt.flatten(),
            eta=jets[two_jets,1].eta.flatten(),
            phi=jets[two_jets,1].phi.flatten(),
            mass=jets[two_jets,1].mass.flatten(),
            qgl=jets[two_jets,1].qgl.flatten()
        )
        
        dijet_pairs = jets[two_jets, 0:2]
        
        dijet_mask = np.zeros(len(event_weight))
        dijet_mask[two_jets] = 2
        dijet_jca = JaggedCandidateArray.candidatesfromcounts(
            dijet_mask,
            pt=dijet_pairs.pt.content,
            eta=dijet_pairs.eta.content,
            phi=dijet_pairs.phi.content,
            mass=dijet_pairs.mass.content,
        )
        
        dijet = dijet_jca.distincts()
        dijet = dijet.p4.sum()

        dijet_deta = np.full(len(event_weight), -999.)
        dijet_deta[two_jets] = abs(jet1[two_jets].eta - jet2[two_jets].eta)
        
        dijet_dphi = np.full(len(event_weight), -999.)
        dijet_dphi[two_jets] = abs(jet1[two_jets].p4.delta_phi(jet2[two_jets].p4))

        vbf_cut = (dijet.mass>400)&(dijet_deta>2.5)
        category[two_jets&(~vbf_cut)] = 'ggh_2j'
        category[two_jets&vbf_cut] = 'vbf'
        
        if self.timer:
            self.timer.add_checkpoint("Computed jet variables")
        
        assert(np.count_nonzero(category)==category.shape[0]), "Not all events have been categorized!"
        channels = list(set(category))
        
#----------------------------------------------------------------------------#
        # flatten variables where exactly one value per event expected
        variable_map = {
            'dimuon_mass': dimuon_mass.flatten(),
            'dimuon_pt': dimuon_pt.flatten(),
            'dimuon_eta': dimuon_eta.flatten(),
            'dimuon_phi': dimuon_phi.flatten(),
            'dimuon_dEta': abs(mu1.eta - mu2.eta).flatten(),
            'dimuon_dPhi': abs(mu1.p4.delta_phi(mu2.p4)).flatten(),
            
            'mu1_pt': mu1.pt.flatten(),
            'mu1_eta': mu1.eta.flatten(),
            'mu1_phi': mu1.phi.flatten(),
            'mu1_iso': mu1.pfRelIso04_all.flatten(),

            'mu2_pt': mu2.pt.flatten().flatten(),
            'mu2_eta': mu2.eta.flatten().flatten(),
            'mu2_phi': mu2.phi.flatten().flatten(),
            'mu2_iso': mu2.pfRelIso04_all.flatten(),
            
            'jet1_pt': jet1.pt,
            'jet1_eta': jet1.eta,
            'jet1_phi': jet1.phi,
            'jet1_qgl': jet1.qgl,

            'jet2_pt': jet2.pt,
            'jet2_eta': jet2.eta,
            'jet2_phi': jet2.phi,
            'jet2_qgl': jet2.qgl,
     
            'jj_mass': dijet.mass,
            'jj_pt': dijet.pt,
            'jj_eta': dijet.eta,
            'jj_phi': dijet.phi,
            'jj_dEta': dijet_deta,
            'jj_dPhi': dijet_dphi,      
 
            'njets': jets.counts.flatten(),
        
            'npv': df.PV.npvsGood.flatten(),
            'met': df.MET.pt.flatten(),
            'event': df.event.flatten(),
            'event_weight': event_weight,
        }

        # Evaluate DNN 

        if self.evaluate_dnn:
            do_dnn_timer = True
            if do_dnn_timer:
                timer_dnn = Timer('dnn evaluation')
                timer_dnn.update()
                
            training_features = ['dimuon_mass', 'dimuon_pt', 'dimuon_eta', 'dimuon_dEta', 'mu1_pt', 'mu2_pt']
            n_rows = len(dimuon_mass.flatten())
            df_for_dnn = pd.DataFrame(columns=training_features)
                
            for tf in training_features:
                feature_column = variable_map[tf]
                assert(n_rows==len(feature_column))
                df_for_dnn[tf] = feature_column

            if do_dnn_timer:
                timer_dnn.add_checkpoint("Filled input dataframe")                
                
            from keras.models import load_model
            import keras.backend as K
            import tensorflow as tf
            config = tf.ConfigProto()
            config.intra_op_parallelism_threads=1
            config.inter_op_parallelism_threads=1
            K.set_session(tf.Session(config=config))
            
            if do_dnn_timer:
                timer_dnn.add_checkpoint("Created TF session")   

            # BOTTLENECK: can't load model outside of a worker
            # https://github.com/keras-team/keras/issues/9964
            dnn_model = load_model('output/trained_models/test.h5')
            
            if do_dnn_timer:
                timer_dnn.add_checkpoint("Loaded model")   
            
            dnn_score = dnn_model.predict(df_for_dnn[training_features]).flatten()
            variable_map['dnn_score'] = dnn_score
            
            if do_dnn_timer:
                timer_dnn.add_checkpoint("Made predictions")
                timer_dnn.summary()
            
            if self.timer:
                self.timer.add_checkpoint("Evaluated DNN")



        #################### Fill outputs ####################
        #------------------ Binned outputs ------------------#  
        for vname, expression in variable_map.items():
            regions = get_regions(variable_map['dimuon_mass'])            
            for cname in channels:
                ccut = (category==cname)
                for rname, rcut in regions.items():
                    if dataset in self.dont_fill[rname]: continue # e.g. dy_m105_160 is only for higgs region
                    if isData and ('h-peak' in rname) and ('dimuon_mass' in vname): continue # blinding
                    value = expression[rcut & ccut]
                    if not value.size: continue # skip empty arrays
                    if isinstance(value, awkward.JaggedArray):
                        # correctly fill arrays with empty elements (for example events with 0 jets)
                        weight = event_weight[rcut & ccut][value.any()]
                    else:
                        weight = event_weight[rcut & ccut]
                    output[vname].fill(**{'dataset': dataset, 'region': rname, 'channel': cname,\
                                         vname: value.flatten(), 'weight': weight})

        output['cutflow']['all events'] += nEvts
    

        
        #----------------- Unbinned outputs -----------------#
        
        if dataset in datasets_to_save_unbin: # don't need unbinned data for all samples
            for v in self.vars_unbin:
                if v not in variable_map.keys(): continue
                for cname in channels:
                    ccut = (category==cname)
                    for rname, rcut in regions.items():
                        if dataset in self.dont_fill[rname]: continue # e.g. dy_m105_160 is only for higgs region
                        if 'z-peak' in rname: continue # don't need unbinned data under Z-peak
                        output[f'{v}_unbin_{dataset}_c_{cname}_r_{rname}'] += processor.column_accumulator(variable_map[v][rcut & ccut])
    
        if self.timer:
            self.timer.add_checkpoint("Filled outputs")
            
        if self.timer:
            self.timer.summary()

        return output
    
    def postprocess(self, accumulator):
        return accumulator

Option 1: Iterative executor
===


In [ ]:
from coffea.processor.executor import iterative_executor
# from python.processor_lite import DimuonProcessorLite
# For debugging purposes

tstart = time.time() 
# output = processor.run_uproot_job(fileset, 'Events', DimuonProcessorLite(), iterative_executor, executor_args={'nano': True})
output = processor.run_uproot_job(fileset, 'Events', DimuonProcessor(do_roccor=True,\
                                                                     evaluate_dnn=True,\
                                                                     do_timer=True),\
                                        iterative_executor, executor_args={'nano': True})
elapsed = time.time() - tstart

print(f"Processed {output['cutflow']['all events']} events")
print(f"Total time: {elapsed} s")
print(f"Rate: {output['cutflow']['all events']/elapsed} events/s")

Option 2: Futures executor
===

In [ ]:
from coffea.processor.executor import futures_executor

tstart = time.time() 
output = processor.run_uproot_job(fileset, 'Events',\
                                  DimuonProcessor(),\
                                  futures_executor,\
                                  executor_args={'nano': True, 'workers':8})
elapsed = time.time() - tstart

print(f"Processed {output['cutflow']['all events']} events")
print(f"Total time: {elapsed} s")
print(f"Rate: {output['cutflow']['all events']/elapsed} events/s")

out_path = "output/test_futures.coffea"
util.save(output, out_path)
print(f"Saved output to {out_path}")

Option 3: Dask executor
===


In [ ]:
import pytest
from coffea.processor.executor import dask_executor
import dask

distributed = pytest.importorskip("distributed", minversion="1.28.1")
distributed.config['distributed']['scheduler']['allowed-failures'] = 20
client = distributed.Client(processes=True, dashboard_address=None, n_workers=4, threads_per_worker=1) 

# Works well for small jobs
# Unstable for large jobs (kernel dies)

tstart = time.time() 
output = processor.run_uproot_job(fileset, 'Events',\
                                  DimuonProcessor(evaluate_dnn=False),\
                                  dask_executor,\
                                  executor_args={'nano': True, 'client': client, 'retries':20})
elapsed = time.time() - tstart

print(f"Processed {output['cutflow']['all events']} events")
print(f"Total time: {elapsed} s")
print(f"Rate: {output['cutflow']['all events']/elapsed} events/s")
      
out_path = "output/test_dask.coffea"
util.save(output, out_path)
print(f"Saved output to {out_path}")      

Option 4: Parsl executor
===

In [ ]:
from coffea.processor.executor import parsl_executor
import parsl
from coffea.processor.parsl.detail import (_parsl_initialize, _parsl_stop, _default_cfg)
_parsl_initialize(config=_default_cfg)

# Doesn't work

tstart = time.time() 
output = processor.run_uproot_job(fileset, 'Events', DimuonProcessor(), parsl_executor, executor_args={'nano': True})
elapsed = time.time() - tstart

print(f"Processed {output['cutflow']['all events']} events")
print(f"Total time: {elapsed} s")
print(f"Rate: {output['cutflow']['all events']/elapsed} events/s")

Option 5: Apache Spark
===


NOW IT IS TIME TO START SPARK CLUSTER CONNECTION
---

When using SWAN, click on the 5-point start icon in Jupyter notebook

In [ ]:
import pyspark.sql
from pyarrow.compat import guid
from coffea.processor.spark.detail import _spark_initialize, _spark_stop
from coffea.processor.spark.spark_executor import spark_executor
"""
# NOT needed on SWAN, spark config is offloaded to spark connector

spark_config = pyspark.sql.SparkSession.builder \
    .appName('spark-executor-test-%s' % guid()) \
    .master('local[*]') \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.memory', '4g') \
    .config('spark.sql.execution.arrow.enabled','true') \
    .config('spark.sql.execution.arrow.maxRecordsPerBatch', 200000)

spark = _spark_initialize(config=spark, log_level='WARN', 
                          spark_progress=False, laurelin_version='0.5.1')
"""
partitionsize = 200000
thread_workers = 2

# Doesn't work (no full NanoEvents support)

tstart = time.time() 
# if jobs fail, it might be because some columns are missing from processor._columns
output = processor.run_spark_job(fileset, DimuonProcessor(), spark_executor, 
                                 spark=spark, partitionsize=partitionsize, thread_workers=thread_workers,
                                 executor_args={'file_type': 'edu.vanderbilt.accre.laurelin.Root', 'cache': False, 'nano': True, 'retries': 5}
                                )

elapsed = time.time() - tstart

print(f"Processed {output['cutflow']['all events']} events")
print(f"Total time: {elapsed} s")
print(f"Rate: {output['cutflow']['all events']/elapsed} events/s")

Plotting
===

In [ ]:
def plot_variable(output, inclusive, channel, region, fig, var, gs, weights):
    data_sources = {
        'data': ['data_B','data_C','data_D','data_E','data_F','data_G','data_H']  
    }
    bkg_sources = {
        'DY': ['dy', 'dy_m105_160_amc'],
        'EWK': ['ewk_lljj_mll50_mjj120','ewk_lljj_mll105_160'],
        'TTbar + Single Top':['ttjets_dl', 'st_tw_top', 'st_tw_antitop'],
        'VV + VVV': ['ww_2l2nu', 'wz_3lnu', 'www','wwz','wzz','zzz'],
    }

    if inclusive:
        output_copy = output[var].sum('region')[:,channel].copy()
        output_copy.scale(weights, axis='dataset')
        data = output_copy.sum('channel').group('dataset', hist.Cat("dataset", "Dataset"), data_sources)
        bkg = output_copy.sum('channel').group('dataset', hist.Cat("dataset", "Dataset"), bkg_sources)
        ggh = output_copy['ggh_amcPS'].sum('channel')
        vbf = output_copy['vbf_amcPS'].sum('channel')
    else:
        output_copy = output[var][:,region, channel].copy()
        output_copy.scale(weights, axis='dataset')
        data = output_copy.sum('region').sum('channel').group('dataset', hist.Cat("dataset", "Dataset"), data_sources)
        bkg = output_copy.sum('region').sum('channel').group('dataset', hist.Cat("dataset", "Dataset"), bkg_sources)
        ggh = output_copy['ggh_amcPS'].sum('region').sum('channel')
        vbf = output_copy['vbf_amcPS'].sum('region').sum('channel')
        
    data_is_valid = data.sum(var).sum('dataset').values()

    bkg.axis('dataset').sorting = 'integral' # sort backgrounds by event yields
        
        
    scale_mc_to_data = True
    if scale_mc_to_data and data_is_valid:
        data_int = data.sum(var).sum('dataset').values()[()]
        bkg_int = bkg.sum(var).sum('dataset').values()[()]    
        bkg_sf = data_int/bkg_int
        bkg.scale(bkg_sf)
        
    data_opts = {'color': 'k', 'marker': '.', 'markersize':15}
    stack_fill_opts = {'alpha': 0.8, 'edgecolor':(0,0,0)}
    stack_error_opts = {'label':'Stat. unc.','facecolor':(0,0,0,.4), 'hatch':'', 'linewidth': 0}
    
    # Top panel: Data vs. MC plot
    plt1 = fig.add_subplot(gs[0])
    ax_bkg = hist.plot1d(bkg, ax=plt1, overlay='dataset', overflow='all', stack=True, fill_opts=stack_fill_opts, error_opts=stack_error_opts)
    # draw signal histograms one by one manually because set_prop_cycle didn't work for changing color map
    ax_ggh = hist.plot1d(ggh, overlay='dataset', overflow='all', line_opts={'linewidth':2, 'color':'r'}, error_opts=None)    
    ax_vbf = hist.plot1d(vbf, overlay='dataset', overflow='all', line_opts={'linewidth':2, 'color':'b'}, error_opts=None)    
    
    if data_is_valid:
        ax_data = hist.plot1d(data, overlay='dataset', overflow='all', line_opts=None, error_opts=data_opts)
    plt1.set_yscale('log')
    plt1.set_ylim(0.001, 1e9)
    lbl = hep.cms.cmslabel(plt1, data=True, paper=False, year='2016')
    plt1.set_xlabel('')
    plt1.tick_params(axis='x', labelbottom=False)
    plt1.legend(prop={'size': 'xx-small'})
    
    # Bottom panel: Data/MC ratio plot
    plt2 = fig.add_subplot(gs[1], sharex=plt1)
    if data_is_valid:
        num = data.sum('dataset')
        denom = bkg.sum('dataset')
        hist.plotratio(num=num, ax=plt2,
                    denom=denom,
                    error_opts=data_opts, denom_fill_opts={}, guide_opts={},
                    unc='num')
    
    
    plt2.axhline(1, ls='--')
    plt2.set_ylim([0.5,1.5])    
    plt2.set_ylabel('Data/MC')
    lbl = plt2.get_xlabel()
    lbl = lbl if lbl else var
    if inclusive:
        plt2.set_xlabel(f'{lbl}, inclusive, {channel} channel')
    else:
        plt2.set_xlabel(f'{lbl}, {region}, {channel} channel')
    

In [ ]:
# Prepare things to plot
from cross_sections import cross_sections
import json

mc_datasets = [s for s in fileset.keys() if 'data' not in s]

lumi_weights = {'data':1}
for mc in mc_datasets:
    N = metadata[mc]['sumGenWgts']
    if 'ewk_lljj_mll50_mjj120' in mc:
        xsec = cross_sections[mc]['2016']
    else:
        xsec = cross_sections[mc]
#     print(f"{mc}: xsec:{xsec}, N:{N}")
    lumi_weights[mc] = xsec*lumi / N

json = json.dumps(lumi_weights)
f = open("output/lumi_weights.json","w")
f.write(json)
f.close()

vars_to_plot = []

vars_to_plot += ['dimuon_mass', 'dimuon_pt', 'dimuon_eta', 'dimuon_phi']
# vars_to_plot += ['dimuon_dEta', 'dimuon_dPhi']
# vars_to_plot += ['mu1_pt', 'mu1_eta', 'mu1_phi',  'mu2_pt', 'mu2_eta', 'mu2_phi']
# vars_to_plot += ['jet1_pt', 'jet1_eta', 'jet1_phi', 'jet1_qgl']
# vars_to_plot += ['jet2_pt', 'jet2_eta', 'jet2_phi', 'jet2_qgl']
# vars_to_plot += ['jj_mass', 'jj_pt', 'jj_eta', 'jj_phi']
# vars_to_plot += ['jj_dEta', 'jj_dPhi']
# vars_to_plot += ['njets', 'npv', 'met']
# vars_to_plot += ['dnn_score']

Make plots and put them into a grid
---

Slower, but output looks nicer.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.cms.style.ROOT)
from matplotlib import gridspec
import math
   
out_path = "output/test_dask.coffea"
output = util.load(out_path)
print(f"Loading output from {out_path}")
fig = plt.figure()

regions = ["z-peak", "h-sidebands", "h-peak"]
channels = ["ggh_01j", "ggh_2j", "vbf"]
nplots_x = 4 # number of plots in one row
nplots_y = math.ceil(len(vars_to_plot)*(len(regions)+1)*len(channels) / nplots_x) # number of rows

plotsize=10
ratio_plot_size = 0.25
fig.set_size_inches(nplots_x*plotsize,nplots_y*plotsize*(1+ratio_plot_size))
outer_grid = gridspec.GridSpec(nplots_y, nplots_x, hspace = .3) 
idx = 0
for var in vars_to_plot:
    for c in channels:
        gs = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec = outer_grid[idx], height_ratios=[(1-ratio_plot_size),ratio_plot_size], hspace = .05)
        plot_variable(output, True, c, "", fig, var, gs, lumi_weights)
        idx += 1
        for r in regions:
    #         print(f"Plotting {var} in {r}")
            gs = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec = outer_grid[idx], height_ratios=[(1-ratio_plot_size),ratio_plot_size], hspace = .05)
            plot_variable(output, False, c, r, fig, var, gs, lumi_weights)
            idx += 1